# Gather MITRE ATT&CK software data for analysis
### Input: line-separated text file with list of MITRE ATT&CK software IDs (e.g. S0496)
### Output: Excel file with three tabs: Datasource requirements, Platform requirements and Tactic 


### Import required libraries. Define filepath.

In [ ]:
from attackcti import attack_client
import json
import numpy
import pandas as pd
import numpy as np
import xlsxwriter

inputFile = '/Users/dan.kaiser/OneDrive - LogRhythm, Inc/Documents/ransomware_module/ransom_SoftwareIDs' #Enter the full path and file name of the text file containing the MITRE Software items you would like analyzed
MITREModuleFile = '/Users/dan.kaiser/OneDrive - LogRhythm, Inc/Documents/ransomware_module/MITREModuleIDs' #Enter the full path and file name of the text file containing the MITRE technique IDs that have rules in the LR MITRE KB Module
outputFile = '/Users/dan.kaiser/OneDrive - LogRhythm, Inc/Documents/ransomware_module/technique_coverage.xlsx' ##Enter the full path and file name of the Excel (xlsx) file that you would like as output of this script

### Load the MITRE ATT&CK Enterprise framework

In [ ]:
lift = attack_client()
%time all_enterprise = lift.get_all_enterprise()

### Read list of techniques from a file

In [ ]:
Softwares = []
Softwares = [line.rstrip() for line in open(inputFile)]
MITRERules = []
MITRERules = [line.rstrip() for line in open(MITREModuleFile)]


In [ ]:
for software in Softwares:
    print(software)

In [ ]:
for rule in MITRERules:
    print(rule)

### Generate a list of techniques for each software

In [ ]:
techniqueAnalysis = []
s = []
for software in Softwares:
    for s in all_enterprise['malware']:
        if s['external_references'][0]['external_id'] == software:
                malwareID = s['id']
                for relationship in all_enterprise['relationships']:
                    techniqueListItem = {} 
                    if relationship['source_ref'] == malwareID:
                        techniqueListItem['MalwareName'] = s['name'] 
                        techniqueListItem['MalwareNumber'] = software
                        techniqueListItem['techniqueID'] = relationship['target_ref']
                        for t in all_enterprise['techniques']:
                            if t['id'] == relationship['target_ref']:
                                techniqueListItem['techniqueNumber'] = t['external_references'][0]['external_id']
                                techniqueListItem['techniqueName'] = t['name']
                                for rule in MITRERules:
                                    if rule == t['external_references'][0]['external_id']:
                                        techniqueListItem['DetectionAvailable'] = 'yes'
                                        break
                                    else:
                                        techniqueListItem['DetectionAvailable'] = 'no'
                        techniqueAnalysis.append(techniqueListItem)
print(techniqueAnalysis)

### Load the tactic list into a dataframe

In [ ]:
techniques = pd.DataFrame(techniqueAnalysis)

### Confirm the dataframe is populated by reviewing per-technique counts

In [ ]:
techniques['techniqueNumber'].value_counts()


### Write out the techniques dataframes into an Excel workbook

In [ ]:
with pd.ExcelWriter(outputFile) as writer:
    techniques.to_excel(writer, sheet_name='Techniques')
